In [1]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
import calendar
import datetime 

In [2]:
# date
def get_date(soup):
    date_span = soup.find('span', class_='submitted')
    date = date_span.find('span').get('content')
    date_object = datetime.datetime.strptime(date, '%Y-%m-%dT%H:%M:%S%z')
    return date_object.date()

In [3]:
# title
def get_title(soup):
    title_h1 = soup.find('h1', class_='title')
    return title_h1.text

In [4]:
# main_html
def get_html(soup):
    html_element = soup.find('div', class_='field field-name-body field-type-text-with-summary field-label-hidden')
    return html_element

def get_html_text(soup):
    html_element = get_html(soup)
    html_main = html_element.decode() 
    return html_main

In [5]:
# full_url
def get_url(soup):
    full_url_tag = soup.find("link", rel="canonical")
    if full_url_tag:
        full_url = full_url_tag.get("href")
        return full_url

In [6]:
# text
def get_text(soup):
    html_element = get_html(soup)
    html_div = html_element.find('div', class_='field-item even')
    elements = html_div.find_all(['p', 'ul', 'ol', 'div'])
    text = ''
    for element in elements:
        if element.name == 'div' and element.find('hr'):
            break
        text += element.text
    return text

In [7]:
# data for dataframe
def get_data(soup):
    data = []
    data.append(get_date(soup))
    data.append(get_title(soup))
    data.append(get_url(soup))
    data.append(get_html_text(soup))
    data.append(get_text(soup))
    return data

In [8]:
# url for request
def build_url(days_to_substract):
    today = datetime.datetime.today() - datetime.timedelta(days=days_to_substract)
    month_name = calendar.month_name[today.month].lower()
    url = f'https://www.understandingwar.org/backgrounder/russian-offensive-campaign-assessment-{month_name}-{today.day}-{today.year}'
    return url

In [9]:
# dataframe
def get_dataframe():
    df = pd.DataFrame(columns=['date', 'title', 'full_url', 'main_html', 'main_text'])
    days_to_substract = 0
    while True:
        days_to_substract += 1
        url = build_url(days_to_substract)
        answer = requests.get(url)
        if not answer.status_code == 200: 
            continue
        html_text = answer.text     
        soup = BeautifulSoup(html_text, 'lxml')
        df.loc[0] = get_data(soup)
        return df

In [10]:
data_frame = get_dataframe()
data_frame

,date,title,full_url,main_html,main_text
0,2024-02-29,"Russian Offensive Campaign Assessment, Februar...",/backgrounder/russian-offensive-campaign-asses...,"<div class=""field field-name-body field-type-t...","Russian Offensive Campaign Assessment, Februar..."


In [12]:
# to save dataframe as csv file
# data_frame.to_csv('yesterday_data.csv', index=False)